In [1]:
%%capture
%run regression.ipynb

In [2]:
%pip install -q tensorflow

Note: you may need to restart the kernel to use updated packages.


In [3]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

In [4]:
# prepare inverse data (nm -> V)
# Input: current positions (x_out, z_out)
# Output: voltage windows that led to those positions
X_inverse = xz_data_out  # (N, 2) - positions
y_inverse = xz_data_in   # (N, 6) - voltage windows
X_train, X_test, y_train, y_test = train_test_split(
     X_inverse, y_inverse, test_size=0.2, random_state=42
)


WINDOW_LEN = 3 # using the same window length as in regression
input_dim = X_inverse.shape[1]  # 2 (x and z positions)
output_dim = y_inverse.shape[1]  # 6 (3 x-voltage + 3 z-voltage windows)
print(input_dim, output_dim)

2 6


In [5]:
# try a simple LinearRegression model first
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
y_pred_lin = lin_reg.predict(X_test)
r2_lin = r2_score(y_test, y_pred_lin)
r2_lin_weighted = r2_score(y_test, y_pred_lin, multioutput="variance_weighted")
print(f"Linear Regression R2: {r2_lin:.4f}")
print(f"Linear Regression R2 variance weighted: {r2_lin_weighted:.4f}")

Linear Regression R2: 0.3770
Linear Regression R2 variance weighted: 0.3924


The feedforward (voltage -> position) R2 score came out to **0.74** (which is good).
The inverse of this (position -> voltage) gives us an R2 score of **0.39**. This means the inverse mapping is fundamentally harder.

In [6]:
# scale the data for the NN
X_scaled = StandardScaler().fit_transform(X_inverse)
y_scaled = StandardScaler().fit_transform(y_inverse)

X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled = train_test_split(
    X_scaled, y_scaled, test_size=0.2, random_state=42
)

In [7]:
# APPROACH 1 - Training a NN with just positions and predicting the voltage window.

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(y_train_scaled.shape[1])
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

history = model.fit(
    X_train_scaled, y_train_scaled,
    validation_split=0.2,
    epochs=500,
    batch_size=128,
    verbose=0,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=30,
            restore_best_weights=True
        ),
    ]
)

y_pred = model.predict(X_test_scaled, verbose=0)
r2 = r2_score(y_test_scaled, y_pred)
r2_weighted = r2_score(y_test_scaled, y_pred, multioutput="variance_weighted")

print(f"Overall R2 score: {r2:.4f}")
print(f"Overall R2 score weighted: {r2_weighted:.4f}")

Overall R2 score: 0.4188
Overall R2 score weighted: 0.4192
